In [2]:
import os
import pickle
import streamlit as st
from dotenv import load_dotenv
from langchain.agents import Tool
from langchain.tools import WikipediaQueryRun
from langchain.schema.document import Document
from langchain.chains import RetrievalQA
from langchain.agents.agent_types import AgentType
from langchain.agents.initialize import initialize_agent
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.utilities import WikipediaAPIWrapper 
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain_community.vectorstores import DocArrayInMemorySearch

In [133]:
# loading API keys from env
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [134]:
# loading model and defining embedding
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')
embeddings = OpenAIEmbeddings()

In [135]:
# setting up info retreival from Wikipedia pages (1st knowledge source)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
prompt = "When did WHO declare an end of COVID-19 emergency?"
wiki_output = wikipedia.run(prompt)

In [136]:
# fragmegting the document content to fit in the number of token limitations
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
new_docs = [Document(page_content=sent) for sent in wiki_output.split('\n')]

# splitted_output = text_splitter.split_documents(new_doc)
data_set = DocArrayInMemorySearch.from_documents(new_docs, embedding=embeddings)

In [137]:
# retreiving the llm response using user query

qa = RetrievalQA.from_chain_type(
    llm =llm,
    chain_type="stuff",
    retriever = data_set.as_retriever(),
    verbose=True,
)

In [138]:
print(qa(prompt)['result'])



> Entering new RetrievalQA chain...

> Finished chain.
The World Health Organization (WHO) ended the Public Health Emergency of International Concern (PHEIC) for COVID-19 on 5 May 2023.


In [139]:
# Loading research paper from web source (2nd knoledge source)
loader = PyPDFLoader("./2312.10997v5.pdf")
docs = loader.load()

# fragmegting the document content to fit in the number of token limitations
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
splits = text_splitter.split_documents(docs)

# load the 
data_set = DocArrayInMemorySearch.from_documents(documents=splits, embedding=embedings)

In [140]:
# retreiving the llm response using user query

qa = RetrievalQA.from_chain_type(
    llm =llm,
    chain_type="stuff",
    retriever = data_set.as_retriever(),
    verbose=True,
)

In [141]:
# get query from U/I now
prompt = "What is 'Advanced RAG' for Large Language Models?"
result = qa(prompt)
print(result['result'])



> Entering new RetrievalQA chain...

> Finished chain.
'Advanced RAG' for Large Language Models refers to the development trajectory of Retrieval-Augmented Generation (RAG) in the era of large models. Initially, RAG focused on enhancing language models by incorporating additional knowledge through Pre-Training Models (PTM). The stages of involving RAG mainly include pre-training, fine-tuning, and inference. Researchers have been exploring ways to enhance language models by leveraging the powerful in-context learning abilities of large language models and integrating more with the fine-tuning of these models. Advanced RAG involves the integration of state-of-the-art technologies in each of these critical components to enhance the performance of RAG systems.


In [142]:
# combining two RAG knoledge sources together for better performance
from langchain.agents.initialize import initialize_agent
wiki_tool = Tool(
    name = "wikipedia",
    func = wikipedia.run,
    description = "A useful tool to search internet for the requested information",
)

docsearch_tool = Tool(
    name = "docsearch",
    func = qa.run,
    description = "A tool to search information from the pool of documents",
)

agent = initialize_agent(
    tools = [wiki_tool, docsearch_tool],
    llm = llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = False,
    handle_pasring_errors = True,
)
print(agent.invoke("When did WHO declares an end of COVID-19 emergency?")['output']) 
print(agent.invoke("What is 'Naive RAG' for Large Language Models?")['output'])

The World Health Organization (WHO) declared an end to the COVID-19 emergency on 5 May 2023.


> Entering new RetrievalQA chain...

> Finished chain.
'Naive RAG' for Large Language Models is a method that relies directly on the user's original query for retrieval, but faces challenges with imprecise queries and language complexity.


In [3]:
# exporting the program termination indicator
fend = open('./notifications/PROG EXIT.txt','wb')
pickle.dump(123, fend)